In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install hazm
!pip install --upgrade git+https://github.com/GRAAL-Research/poutyne.git@dev #install poutyne
!pip install --upgrade colorama #install colorama
!pip install --upgrade pymagnitude-light #install pymagnitude-light
!matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 76.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 27.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394484 sha256=872cf873320d7655096f64e3f23f39c7a249ece53975502bb54826e903778c66
  Stored in directory: /root/.cache/pip/wheels/19/1d/3a/0a8c14c30132b4f9ffd796efbb6746f15b3d6bcfc1055a9346
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp38-cp38-linux_x86_64.whl size=180772 sha256=8112ef910275bf708793192b69abfb9f197c85914f938986b5a2878a6113431b
  Stored in directory: /root/.cache/pip/wheels/3c/d8/9f/59fd78b2b7d1e9ffcb68fb6de80c2e7c20b804c8cbc4d8fc23
Successfully built nltk libwapiti
  Attempting unin

<div style='direction:rtl; font-family: "B Lotus"; font-size: 16px'>
در این قسمت، داده‌های برچسب گذاری شده را ابتدا نرمال سازی می‌کنیم و بعد توکن‌ها 
را به دست می‌آوریم و آن را به فرمتی که برای تزریق به شبکه مورد نیاز است در می‌آوریم.
</div>

In [1]:
import codecs
import math
import numpy as np
import tqdm
import torch
from hazm import *

class Loader:
    path_data = '/content/drive/My Drive/sample.out.txt'
    start_tag_list = ['<c>', '<m>', '<e>']
    end_tag_list = ['</c>', '</m>', '</e>']
    none_tag = '<n>'

    def __init__(self):
        self.normalizer = Normalizer()
        self.lemmatizer = Lemmatizer()

    def create_data_label(self):
        lines = codecs.open(self.path_data, 'r', 'utf-8').readlines()

        data = []
        for sent in lines:
            sent = self.normalizer.normalize(sent)
            token_list = []
            label_list = []
            last_tag = self.none_tag
            for token in word_tokenize(sent):
                if token in self.start_tag_list:
                    last_tag = token
                    continue
                if token in self.end_tag_list:
                    last_tag = self.none_tag
                    continue

                token_list.append(self.lemmatizer.lemmatize(token))
                label_list.append(last_tag)

            tokenToStr = ' '.join([str(elem) for elem in token_list])
            token = (tokenToStr, label_list)
            data.append(token)
        return data

loader = Loader()
data = loader.create_data_label()
print(f'Data[0] = {data[0]}')


Data[0] = ('این مساله باعث شد#شو او را از بانفوذ اعضا جوان حزب ندا ایران در میان اصلاح طلب شناخت#شناس .', ['<c>', '<c>', '<m>', '<n>', '<n>', '<n>', '<n>', '<e>', '<e>', '<e>', '<e>', '<e>', '<e>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>'])


In [2]:
import gzip
import shutil
import warnings

import torch
import torch.nn as nn
import torch.optim as optim
from poutyne import set_seeds
from poutyne.framework import Experiment
from pymagnitudelight import Magnitude
from torch.nn.functional import cross_entropy
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence, pad_sequence
from torch.utils.data import DataLoader

<div style='direction:rtl; font-family: "B Lotus"; font-size: 16px'>
در این قسمت مشخصات شبکه را تنظیم می‌کنیم. از ال‌اس‌تی‌ام یک لایه و یک طرفه استفاده می‌کنیم. خروجی ال‌اس‌تی‌ام ۳۰۰ بعدی است که باید به فضای ۴ بعدی که تعداد تگ‌هاست مپ شود. سپس  تنظیمات طول ایپاک و سایز بچ را انجام می‌دهیم.

In [3]:
dimension = 300
num_layer = 1
bidirectional = False

lstm_network = nn.LSTM(
    input_size=dimension,
    hidden_size=dimension,
    num_layers=num_layer,
    bidirectional=bidirectional,
    batch_first=True,
)

input_dim = dimension 
tag_dimension = 4

fully_connected_network = nn.Linear(input_dim, tag_dimension)

device = torch.device("cuda:0")

batch_size = 64
lr = 0.1

epoch_number = 10
set_seeds(42)

داده‌ها به تست و ارزیابی و آموزش تقسیم می‌شوند و یک نمونه خروجی از هر کدام می‌بینیم.

In [4]:
np.random.shuffle(data)
train_size = math.ceil(len(data) * 0.8)
val_size = math.ceil(len(data) * 0.1)
test_size = len(data) - train_size - val_size

train_data = data[0:train_size]
valid_data = data[train_size:train_size + val_size]
test_data = data[train_size + val_size:train_size + val_size + test_size]
print(train_data[0:1])
print(valid_data[0:1])
print(test_data[0:1])


[('در گذشته تی تی را اغلب در کنار رودخانه و همراه شنا کردن انجام داد#ده تا گرفتن افراد ( به دلیل لباس نداشتن و لیز بودن ) سخت شد#شو', ['<n>', '<n>', '<c>', '<c>', '<c>', '<c>', '<c>', '<c>', '<c>', '<c>', '<c>', '<c>', '<c>', '<c>', '<c>', '<m>', '<e>', '<e>', '<e>', '<e>', '<e>', '<e>', '<e>', '<e>', '<e>', '<e>', '<e>', '<e>', '<n>'])]
[('کاستاریکا هم چهار امتیاز داشت اما به دلیل تفاضل گل کمتر حذف شد#شو .', ['<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<m>', '<m>', '<c>', '<c>', '<c>', '<e>', '<e>', '<n>'])]
[('ورزش متابولیسم را افزایش داد#ده که منجر به کاهش وزن شد#شو .', ['<c>', '<e>', '<n>', '<m>', '<m>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>'])]


جاسازی توکن‌ها به کمک فست تکست انجام می‌شود و تگ‌ها هم شماره ۰ تا ۳ دریافت می‌کنند..

In [8]:
!pip install --upgrade fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.3-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=4401947 sha256=eaf45d67dd9b05eda19d9c3eeb73ac6588467b4bb1834b49d0c61e90e5c9c716
  Stored in directory: /root/.cache/pip/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built fasttext


In [5]:
import contextlib
import os
import pickle
import re
import sys
from io import TextIOBase

import fasttext
import fasttext.util
import requests

class LookForProgress(TextIOBase):
    def __init__(self, stdout):
        self.stdout = stdout
        self.regex = re.compile(r'([0-9]+(\.[0-9]+)?%)', re.IGNORECASE)

    def write(self, o):
        res = self.regex.findall(o)
        if len(res) != 0:
            print(f"\r{res[-1][0]}", end='', file=self.stdout)


class EmbeddingVectorizer:
    def __init__(self):
        with contextlib.redirect_stdout(LookForProgress(sys.stdout)):
            fasttext.util.download_model('fa', if_exists='ignore')
        self.embedding_model = fasttext.load_model("./cc.fa.300.bin")

    def __call__(self, sent):

        embeddings = []
        for word in sent.split():
            embeddings.append(self.embedding_model[word])
        return embeddings


embedding_vectorizer = EmbeddingVectorizer()

In [6]:
class DatasetBucket:
    def __init__(self, dataset, embedding_model, predict=False):
        self.data = dataset
        self.embedding_model = embedding_model
        self.predict = predict
        self.tags_set = {
            '<n>': 0,
            '<c>': 1,
            '<m>': 2,
            '<e>': 3,

        }

    def __len__(self):
        # for the dataloader
        return len(self.data)

    def __getitem__(self, item):
        data = self.data[item]

        if not self.predict:
            address = data[0]
            address_vector = self.embedding_model(address)

            tags = data[1]
            idx_tags = self._convert_tags_to_idx(tags)
            return address_vector, idx_tags

        address_vector = self.embedding_model(data)
        return address_vector

    def _convert_tags_to_idx(self, tags):
        idx_tags = []
        for tag in tags:
            idx_tags.append(self.tags_set[tag])
        return idx_tags

train_dataset_vectorizer = DatasetBucket(train_data, embedding_vectorizer)
valid_dataset_vectorizer = DatasetBucket(valid_data, embedding_vectorizer)
test_dataset_vectorizer = DatasetBucket(test_data, embedding_vectorizer)

برای یکسان کردن طول جملات از پدینگ استفاده می‌شود.

In [7]:
def pad_collate_fn(batch):

    sequences_vectors, sequences_labels, lengths = zip(
        *[
            (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
            for (seq_vectors, labels) in sorted(
                batch, key=lambda x: len(x[0]), reverse=True
            )
        ]
    )

    lengths = torch.LongTensor(lengths)

    padded_sequences_vectors = pad_sequence(
        sequences_vectors, batch_first=True, padding_value=0
    )
    pack_padded_sequences_vectors = pack_padded_sequence(
        padded_sequences_vectors, lengths.cpu(), batch_first=True
    )  

    padded_sequences_labels = pad_sequence(
        sequences_labels, batch_first=True, padding_value=-100
    )

    return pack_padded_sequences_vectors, padded_sequences_labels

In [8]:
train_loader = DataLoader(
    train_dataset_vectorizer,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=pad_collate_fn,
    num_workers=4,
)
valid_loader = DataLoader(
    valid_dataset_vectorizer,
    batch_size=batch_size,
    collate_fn=pad_collate_fn,
    num_workers=4,
)
test_loader = DataLoader(
    test_dataset_vectorizer,
    batch_size=batch_size,
    collate_fn=pad_collate_fn,
    num_workers=2,
)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


در این قسمت آموزش مدل انجام می‌شود.

In [9]:
class RecurrentNet(nn.Module):
    def __init__(self, lstm_network, fully_connected_network):
        super().__init__()
        self.hidden_state = None

        self.lstm_network = lstm_network
        self.fully_connected_network = fully_connected_network

    def forward(self, packed_sequences_vectors):
        lstm_out, self.hidden_state = self.lstm_network(packed_sequences_vectors)
        lstm_out, _ = pad_packed_sequence(lstm_out, batch_first=True)

        tag_space = self.fully_connected_network(lstm_out)
        return tag_space.transpose(-1, 1)  # We need to transpose since it's a sequence


full_network = RecurrentNet(lstm_network, fully_connected_network)

In [20]:

from poutyne import set_seeds, Model
optimizer = optim.SGD(full_network.parameters(), lr)
loss_function = nn.CrossEntropyLoss()
model = Model(
    full_network,
    optimizer,
    loss_function,
     batch_metrics=['accuracy','f1' ,'recall' ],
    device=device,
)

model.fit_generator(train_loader, valid_loader, epochs=30)

test_loss, test_acc = model.evaluate_generator(test_loader)



<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torc

Epoch:  1/30 Step: 41/42  97.62% |███████████████████▌|ETA: 0.04s loss: 0.970550 acc: 60.940033 fscore_macro: 0.593906 recall_macro: 0.580636

<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torc

Epoch:  1/30 Train steps: 42 Val steps: 6 4.21s loss: 0.973052 acc: 59.089329 fscore_macro: 0.578760 recall_macro: 0.564827 val_loss: 1.372760 val_acc: 47.127765 val_fscore_macro: 0.446337 val_recall_macro: 0.453438


<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torc

...

Epoch: 30/30 Step: 37/42  88.10% |█████████████████▌  |ETA: 0.39s loss: 0.876796 acc: 63.600628 fscore_macro: 0.650338 recall_macro: 0.650001

<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torc

Epoch: 30/30 Train steps: 42 Val steps: 6 5.84s loss: 0.904148 acc: 62.391807 fscore_macro: 0.622545 recall_macro: 0.611886 val_loss: 1.079405 val_acc: 57.374043 val_fscore_macro: 0.563906 val_recall_macro: 0.552548


<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))


Test steps: 6 0.81s test_loss: 1.047176 test_acc: 58.307146 test_fscore_macro: 0.569567 test_recall_macro: 0.555706                                               


روی یک داده‌ی نمونه خروجی می‌گیریم تا تگ‌های تولید شده را مشاهده کنیم.

In [21]:
predict_data = ["چون خوب درس نخوانده بود نمره‌ی خوبی نگرفت" , "علی درگیر بیماری بود و در نهایت به دلیل سرطان درگذشت"]
predict_data_vectorized = DatasetBucket(predict_data, embedding_vectorizer, predict=True)

def pad_collate_fn_predict(batch):

    sequences_vectors, lengths = zip(
        *[
            (torch.FloatTensor(seq_vectors), len(seq_vectors))
            for seq_vectors in sorted(batch, key=lambda x: len(x), reverse=True)
        ]
    )

    lengths = torch.LongTensor(lengths)

    padded_sequences_vectors = pad_sequence(sequences_vectors, batch_first=True, padding_value=0)
    pack_padded_sequences_vectors = pack_padded_sequence(padded_sequences_vectors, lengths.cpu(), batch_first=True)

    return pack_padded_sequences_vectors


predict_loader = DataLoader(
    predict_data_vectorized,
    batch_size=batch_size,
    collate_fn=pad_collate_fn_predict,
    num_workers=2,
)
predictions = model.predict_generator(predict_loader, concatenate_returns=False)
predictions
idx_predictions = []
for batch in predictions:
    idx_predictions.extend(batch.argmax(axis=1).tolist()) 
idx_to_tags = {
     0:'<n>',
     1:'<c>',
     2:'<m>',
     3:'<e>',
     
          }

tags_predictions = []
for address in idx_predictions:
    tags_predictions.append([idx_to_tags.get(tag) for tag in address])

<ipython-input-21-70e75ced0b35>:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), len(seq_vectors))


Prediction steps: 1 0.48s                                          


In [22]:
token = predict_data[0].split()
count = 0
for t in token :
  print( '(' + t + ',' + tags_predictions[0][count]+ ')')
  count +=1
print('________________________________________________-')
token = predict_data[1].split()
count = 0
for t in token :
  print( '(' + t + ',' + tags_predictions[0][count]+ ')')
  count +=1

(چون,<n>)
(خوب,<n>)
(درس,<n>)
(نخوانده,<n>)
(بود,<n>)
(نمره‌ی,<n>)
(خوبی,<n>)
(نگرفت,<m>)
________________________________________________-
(علی,<n>)
(درگیر,<n>)
(بیماری,<n>)
(بود,<n>)
(و,<n>)
(در,<n>)
(نهایت,<n>)
(به,<m>)
(دلیل,<m>)
(سرطان,<e>)
(درگذشت,<e>)


شبکه را دو لایه و دو طرفه می‌کنیم

In [23]:
dimension = 300
num_layer = 2
bidirectional = True

lstm_network = nn.LSTM(
    input_size=dimension,
    hidden_size=dimension,
    num_layers=num_layer,
    bidirectional=bidirectional,
    batch_first=True,
)

input_dim = dimension * 2  # since bidirectional

fully_connected_network = nn.Linear(input_dim, tag_dimension)

full_network_bi_lstm = RecurrentNet(lstm_network, fully_connected_network)

In [14]:
optimizer = optim.SGD(full_network_bi_lstm.parameters(), lr)
loss_function = nn.CrossEntropyLoss()

model2 = Model(
    full_network_bi_lstm,
    optimizer,
    loss_function,
     batch_metrics=['accuracy','f1' ,'recall' ,'precision' ],
    device=device,
)

model2.fit_generator(train_loader, valid_loader, epochs=30)

test_loss, test_acc = model2.evaluate_generator(test_loader)


<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torc

Epoch:  1/30 Step: 40/42  95.24% |███████████████████ |ETA: 0.10s loss: 1.295866 acc: 36.671684 fscore_macro: 0.134160 recall_macro: 0.250000

<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torc

Epoch:  1/30 Train steps: 42 Val steps: 6 4.45s loss: 1.313303 acc: 38.334755 fscore_macro: 0.166493 recall_macro: 0.249930 val_loss: 1.283279 val_acc: 40.419791 val_fscore_macro: 0.143881 val_recall_macro: 0.250000


<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torc

...

Epoch: 30/30 Step: 38/42  90.48% |██████████████████  |ETA: 0.20s loss: 1.135534 acc: 51.525425 fscore_macro: 0.403561 recall_macro: 0.431493

<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torc

Epoch: 30/30 Train steps: 42 Val steps: 6 5.14s loss: 1.141295 acc: 51.644377 fscore_macro: 0.391297 recall_macro: 0.406962 val_loss: 1.157388 val_acc: 48.711166 val_fscore_macro: 0.379227 val_recall_macro: 0.407894


<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-7-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))


Test steps: 6 1.49s test_loss: 1.149401 test_acc: 49.942586 test_fscore_macro: 0.391272 test_recall_macro: 0.419080                                               


و مجدد خروجی را چک می‌کنیم.

In [15]:
predict_data = ["چون خوب درس نخوانده بود نمره‌ی خوبی نگرفت" , "علی درگیر بیماری بود و در نهایت به دلیل سرطان درگذشت"]
predict_data_vectorized = DatasetBucket(predict_data, embedding_vectorizer, predict=True)

def pad_collate_fn_predict(batch):

    sequences_vectors, lengths = zip(
        *[
            (torch.FloatTensor(seq_vectors), len(seq_vectors))
            for seq_vectors in sorted(batch, key=lambda x: len(x), reverse=True)
        ]
    )

    lengths = torch.LongTensor(lengths)

    padded_sequences_vectors = pad_sequence(sequences_vectors, batch_first=True, padding_value=0)
    pack_padded_sequences_vectors = pack_padded_sequence(padded_sequences_vectors, lengths.cpu(), batch_first=True)

    return pack_padded_sequences_vectors


predict_loader = DataLoader(
    predict_data_vectorized,
    batch_size=batch_size,
    collate_fn=pad_collate_fn_predict,
    num_workers=2,
)
predictions = model.predict_generator(predict_loader, concatenate_returns=False)
predictions
idx_predictions = []
for batch in predictions:
    idx_predictions.extend(batch.argmax(axis=1).tolist()) 
idx_to_tags = {
     0:'<n>',
     1:'<c>',
     2:'<m>',
     3:'<e>',
     
          }

tags_predictions = []
for address in idx_predictions:
    tags_predictions.append([idx_to_tags.get(tag) for tag in address])

<ipython-input-15-70e75ced0b35>:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), len(seq_vectors))


Prediction steps: 1 0.46s                                          


In [16]:
token = predict_data[0].split()
count = 0
for t in token :
  print( '(' + t + ',' + tags_predictions[0][count]+ ')')
  count +=1
print('________________________________________________-')
token = predict_data[1].split()
count = 0
for t in token :
  print( '(' + t + ',' + tags_predictions[0][count]+ ')')
  count +=1

(چون,<n>)
(خوب,<c>)
(درس,<c>)
(نخوانده,<c>)
(بود,<n>)
(نمره‌ی,<n>)
(خوبی,<n>)
(نگرفت,<e>)
________________________________________________-
(علی,<n>)
(درگیر,<c>)
(بیماری,<c>)
(بود,<c>)
(و,<n>)
(در,<n>)
(نهایت,<n>)
(به,<e>)
(دلیل,<e>)
(سرطان,<e>)
(درگذشت,<e>)
